In [581]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import RobustScaler
import pandas as pd
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_columns', None)

In [582]:
df=pd.read_parquet('../data/processed/df.parquet')
dfd=pd.read_parquet('../data/processed/dfd.parquet')


In [583]:
dfd.columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'df_original', 'id',
       'cut_int', 'color_int', 'clarity_int', 'cut_color_clarity_score',
       'raw_density', 'marketing_carat_int', 'carat_sqr', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'marketing_carat_bad',
       'marketing_carat_good', 'marketing_carat_regular'],
      dtype='object')

In [584]:
#option 1

dfd=dfd.drop(columns=['marketing_carat_int','cut_int','color_int', 'clarity_int'])

# #option 2
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score'])

# #option 3 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z'])

# #option 3 bis espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density'])

# #option 3 bis2 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'x', 'y', 'z'])

# #option 4
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z','raw_density'])

# #option 5 SIN NADA
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density','marketing_carat_bad',
#        'marketing_carat_good', 'marketing_carat_regular'])

#option 5 SIN NADA
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density','marketing_carat_bad',
#        'marketing_carat_good', 'marketing_carat_regular','carat_sqr'])

In [585]:
#option 1

df=df.drop(columns=['cut','color', 'clarity','marketing_carat'])

# #option 2
# df=df.drop(columns=['cut','color', 'clarity','marketing_carat','cut_color_clarity_score'])


In [586]:
df


,carat,depth,table,price,x,y,z,df_original,id,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,marketing_carat_int,carat_sqr
0,1.21,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,4,1,4,31,3.683471,1,1.4641
1,0.32,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,3,3,4,34,3.664419,1,0.1024
2,0.71,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,1,4,5,35,3.755392,1,0.5041
3,0.41,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,2,7,3,39,3.712154,1,0.1681
4,1.02,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,5,4,3,39,3.633558,2,1.0404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,NaN,5.35,5.32,3.30,0,13480.0,5,6,3,45,3.641219,1,0.3249
13481,0.71,62.2,55.0,NaN,5.71,5.73,3.56,0,13481.0,5,2,4,37,3.657370,1,0.5041
13482,0.70,61.6,55.0,NaN,5.75,5.71,3.53,0,13482.0,5,5,5,50,3.623853,1,0.4900
13483,0.70,58.8,57.0,NaN,5.85,5.89,3.45,0,13483.0,3,5,2,32,3.533125,1,0.4900


In [587]:
dfd

,carat,depth,table,price,x,y,z,df_original,id,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular
0,1.21,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,31,3.683471,1.4641,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
1,0.32,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,34,3.664419,0.1024,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.71,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,35,3.755392,0.5041,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,0.41,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,39,3.712154,0.1681,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,1.02,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,39,3.633558,1.0404,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,NaN,5.35,5.32,3.30,0,13480.0,45,3.641219,0.3249,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
13481,0.71,62.2,55.0,NaN,5.71,5.73,3.56,0,13481.0,37,3.657370,0.5041,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
13482,0.70,61.6,55.0,NaN,5.75,5.71,3.53,0,13482.0,50,3.623853,0.4900,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
13483,0.70,58.8,57.0,NaN,5.85,5.89,3.45,0,13483.0,32,3.533125,0.4900,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


## Split

In [597]:


df_final=dfd

data=df_final.loc[df['df_original']==1]

############## FILAS


data=data.drop_duplicates(subset=data.columns, keep='first')

data['depth_calc'] = data['z'] / ((data['x'] + data['y'])/2)*100
data['rest']=data['depth_calc'] - data['depth'] 
filter_rest=(data['rest']<-1) | (data['rest']>1)
data=data.loc[~filter_rest]
data=data.drop(columns=['depth_calc','rest'],axis=1)

filter_xyz=(data['x']<0.2) | (data['y']<0.2) | (data['z']<0.2)
data=data.loc[~filter_xyz ]



df_predict=df_final.loc[df['df_original']==0]

X=data.drop(columns=['price','df_original','id'])
Y=data['price']

X_predict=df_predict.drop(columns=['price','df_original','id'])


# rs = RobustScaler() 
# X = pd.DataFrame(rs.fit_transform(X), columns= X.columns)
# X_predict = pd.DataFrame(rs.fit_transform(X_predict), columns= X_predict.columns)

X.to_parquet('../data/processed/X.parquet')
Y.to_parquet('../data/processed/Y.parquet')
X_predict.to_parquet('../data/processed/X_predict.parquet')
#OUTLIERSSSSSSSSSS


print(data.shape)
print(df_predict.shape)
print(X.shape)
print(X_predict.shape)
# print(X.describe())
# print(X_predict.describe())


(40292, 16)
(13485, 16)
(40292, 13)
(13485, 13)


## Model Generator

In [598]:
# arboles  de decision max_depth probar min sample split


estimators=[ ('LinearRegression', LinearRegression(fit_intercept=True)),
            ('DecisionTreeRegressor', DecisionTreeRegressor(random_state=0))]



machine = [LinearRegression(fit_intercept=True),
             DecisionTreeRegressor(random_state=0),
           StackingRegressor(estimators=estimators,final_estimator= LinearRegression(n_jobs = -1),cv= 5 , verbose = 2 , passthrough = True)]  

# machine = [  LinearRegression(fit_intercept=True)  ]

for x in machine:
    print(f'Model in action: {x}\n')
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.85, random_state=32)
    
######################           ENTRENAR AL FINAL CON TODO


    
#     features = RFECV(x)
#     features= features.fit(X_train, Y_train)
#     features_columns=[X.columns[pos] for pos,item in enumerate(features.support_.tolist()) if item==False]
#     X_train=X_train.drop(columns=features_columns)
#     X_test=X_test.drop(columns=features_columns)
#   NO RESULETOOOOOOOOOOOOOOOOOOOX_tespredictttttttttttttttt=X_test.drop(columns=features_columns)
#     print(f'Delete columns: {len(X.columns)-features.n_features_}  {features_columns} ')
        
        
    model = x  
    model.fit(X_train, Y_train)
    predictions_train = model.predict(X_train)#.clip(326,18823)
    predictions_test = model.predict(X_test)#.clip(326,18823)

    print(f' R2 Train: {r2_score(y_true=Y_train, y_pred=predictions_train)}')
    print(f' R2 Test: {r2_score(y_true=Y_test, y_pred=predictions_test)}')
    print(f' RMSE = {(mean_squared_error(Y_test, predictions_test,squared=False))}\n\n')
    









Model in action: LinearRegression()

 R2 Train: 0.9180025093466709
 R2 Test: 0.9178891020684274
 RMSE = 1150.9058279803462


Model in action: DecisionTreeRegressor(random_state=0)

 R2 Train: 0.9999972359273196
 R2 Test: 0.9635874494606347
 RMSE = 766.4166637013274


Model in action: StackingRegressor(cv=5,
                  estimators=[('LinearRegression', LinearRegression()),
                              ('DecisionTreeRegressor',
                               DecisionTreeRegressor(random_state=0))],
                  final_estimator=LinearRegression(n_jobs=-1), passthrough=True,
                  verbose=2)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


 R2 Train: 0.9951513285135202
 R2 Test: 0.9683477541651426
 RMSE = 714.564857400896




[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.3s finished


In [599]:
dfX_test=pd.DataFrame(X_test)
dfY_test=pd.DataFrame(Y_test)
dfpredictions_test=pd.DataFrame(predictions_test)
dftest=pd.concat([dfX_test.reset_index(drop=True),dfY_test.reset_index(drop=True),dfpredictions_test.reset_index(drop=True)], axis=1) 
dftest.rename(columns={0: 'price_predict'},inplace=True)
dftest['Diff']=dftest['price']-dftest['price_predict']

print(dftest['Diff'].describe())
dftest.loc[dftest['Diff']>2000].describe()

count    6044.000000
mean       -1.756358
std       714.621820
min     -6486.805438
25%      -216.154551
50%         5.890280
75%       212.982704
max      6952.398449
Name: Diff, dtype: float64


,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,marketing_carat_int,carat_sqr,price,price_predict,Diff
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.00000,99.000000,99.000000
mean,1.711818,61.833333,57.730303,7.612727,7.595657,4.702525,3.757576,4.040404,3.909091,39.030303,3.697738,1.212121,3.089791,14936.40404,12116.650516,2819.753524
std,0.401368,1.557143,2.238096,0.665461,0.651928,0.420210,1.134711,1.420830,1.690857,9.847256,0.066257,0.457873,1.313592,3329.60935,3140.768996,794.863291
min,0.340000,55.800000,51.000000,4.720000,4.600000,2.600000,1.000000,1.000000,1.000000,22.000000,3.538556,0.000000,0.115600,2160.00000,-213.262654,2001.107482
25%,1.500000,61.150000,56.000000,7.300000,7.275000,4.490000,3.000000,3.000000,3.000000,32.000000,3.651657,1.000000,2.250000,13300.00000,10348.919310,2338.017167
50%,1.710000,62.000000,58.000000,7.700000,7.650000,4.720000,4.000000,4.000000,4.000000,37.000000,3.693289,1.000000,2.924100,15827.00000,12947.100540,2574.579699
75%,2.035000,62.700000,59.000000,8.155000,8.095000,5.030000,5.000000,5.000000,5.000000,46.000000,3.735690,1.000000,4.141250,17511.00000,14506.245016,3075.411915
max,2.510000,67.500000,63.000000,8.870000,8.790000,5.440000,5.000000,7.000000,8.000000,62.000000,3.917709,2.000000,6.300100,18777.00000,16465.214691,6952.398449


In [591]:
# #####################importante  y_predict.clip(326,18823) min y maximo
# scores = cross_val_score(model, 
#                          diamonds[FEATS], 
#                          diamonds[TARGET], 
#                          scoring='neg_root_mean_squared_error', 
#                          cv=5, n_jobs=-1)
#para validarlo pero peta el ordenador

predictions_competition = model.predict(X_predict)#.clip(326,18823)
submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition})
submission['id'] = submission['id'].astype('int64')

submission.to_csv('/home/carpiero/IRONHACK/kaggle/submission.csv', index=False)

PROBA=pd.concat([X_predict,submission], axis=1)



submission


,id,price
0,0,3090.738350
1,1,5808.588369
2,2,10968.512550
3,3,4500.097922
4,4,1723.956782
...,...,...
13480,13480,1669.025641
13481,13481,2464.894257
13482,13482,3264.826688
13483,13483,1939.973014


In [592]:
PROBA.shape

(13485, 34)

In [593]:
PROBA

,carat,depth,table,x,y,z,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular,id,price
0,0.79,62.7,60.0,5.82,5.89,3.67,36,3.767680,0.6241,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,3090.738350
1,1.20,61.0,57.0,6.81,6.89,4.18,38,3.671046,1.4400,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,5808.588369
2,1.57,62.2,61.0,7.38,7.32,4.57,33,3.815639,2.4649,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,2,10968.512550
3,0.90,63.8,54.0,6.09,6.13,3.90,36,3.708953,0.8100,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,3,4500.097922
4,0.50,62.9,58.0,5.05,5.09,3.19,44,3.658655,0.2500,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,4,1723.956782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,45,3.641219,0.3249,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,13480,1669.025641
13481,0.71,62.2,55.0,5.71,5.73,3.56,37,3.657370,0.5041,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,13481,2464.894257
13482,0.70,61.6,55.0,5.75,5.71,3.53,50,3.623853,0.4900,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,13482,3264.826688
13483,0.70,58.8,57.0,5.85,5.89,3.45,32,3.533125,0.4900,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,13483,1939.973014


In [594]:
PROBA.describe()

,carat,depth,table,x,y,z,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular,id,price
count,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000
mean,0.798642,61.739095,57.490337,5.736454,5.739648,3.543474,41.192510,3.675548,0.858149,0.029143,0.092176,0.395328,0.261846,0.221505,0.127994,0.183315,0.175380,0.210382,0.150315,0.103077,0.049537,0.013200,0.034334,0.245903,0.167964,0.156099,0.223211,0.065332,0.093956,0.026696,0.126066,0.847238,6742.000000,3944.613441
std,0.469399,1.435310,2.237109,1.113671,1.128507,0.731005,9.270668,0.132355,1.047192,0.168215,0.289286,0.488939,0.439656,0.415275,0.334096,0.386939,0.380306,0.407595,0.357393,0.304071,0.216993,0.114134,0.182094,0.430637,0.373849,0.362963,0.416414,0.247120,0.291779,0.161200,0.331936,0.359772,3892.928525,3995.406745
min,0.200000,50.800000,51.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-81905.748626
25%,0.400000,61.000000,56.000000,4.730000,4.730000,2.920000,35.000000,3.629435,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3371.000000,997.898980
50%,0.700000,61.900000,57.000000,5.700000,5.720000,3.530000,41.000000,3.670425,0.490000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6742.000000,2460.291117
75%,1.040000,62.500000,59.000000,6.530000,6.530000,4.040000,48.000000,3.714315,1.081600,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,10113.000000,5550.876195
max,5.010000,79.000000,73.000000,10.740000,31.800000,31.800000,68.000000,11.368330,25.100100,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13484.000000,27970.409877


In [595]:
PROBA.loc[(PROBA['price']>19000) | (PROBA['price']<0)]

,carat,depth,table,x,y,z,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular,id,price
70,0.33,61.4,58.0,4.46,4.50,2.75,27,3.587444,0.1089,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,70,-3.882023
274,0.31,60.7,60.0,4.37,4.33,2.64,30,3.723400,0.0961,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,274,-81.136283
298,0.33,62.4,54.0,4.43,4.45,2.77,26,3.625949,0.1089,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,298,-506.689301
311,0.30,63.7,54.0,4.25,4.23,2.70,26,3.708339,0.0900,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,311,-12.755981
418,0.30,62.3,57.0,4.27,4.30,2.67,30,3.671681,0.0900,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,418,-295.815113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,0.30,61.9,60.0,4.27,4.29,2.65,29,3.708015,0.0900,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,13115,-153.861149
13161,0.70,66.5,57.0,5.51,5.40,3.64,16,3.877953,0.4900,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,13161,-89.606630
13254,0.31,61.4,56.0,4.35,4.38,2.68,33,3.642628,0.0961,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,13254,-14.595150
13322,0.32,62.3,57.0,4.44,4.33,2.73,30,3.658202,0.1024,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,13322,-288.793203


In [596]:
PROBA.loc[(PROBA['price']>18000) | (PROBA['price']<0)].describe()

,carat,depth,table,x,y,z,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular,id,price
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.0,128.0,128.000000,128.000000,128.000000,128.000000
mean,0.570234,61.907031,57.579687,4.774844,4.976484,3.163828,29.546875,3.501993,0.765502,0.070312,0.070312,0.281250,0.359375,0.218750,0.015625,0.046875,0.023438,0.085938,0.210938,0.296875,0.320312,0.117188,0.007812,0.382812,0.304688,0.070312,0.101562,0.015625,0.0,0.0,0.039062,0.960938,6317.398438,1680.841995
std,0.666185,1.697529,2.181250,1.455001,2.790744,2.728628,6.901853,0.757046,2.095185,0.256678,0.256678,0.451376,0.481703,0.415023,0.124507,0.212202,0.151883,0.281373,0.409577,0.458676,0.468430,0.322907,0.088388,0.487983,0.462084,0.256678,0.303259,0.124507,0.0,0.0,0.194505,0.194505,3856.790303,9999.718681
min,0.290000,58.100000,53.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.084100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,70.000000,-81905.748626
25%,0.300000,60.975000,56.000000,4.330000,4.327500,2.670000,26.000000,3.607892,0.090000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000,2881.500000,-204.041730
50%,0.320000,61.800000,58.000000,4.390000,4.405000,2.710000,30.000000,3.658456,0.102400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000,6646.500000,-95.087408
75%,0.387500,62.700000,59.000000,4.612500,4.627500,2.812500,33.000000,3.717087,0.150325,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000,9480.500000,-28.834132
max,3.650000,68.400000,62.000000,9.530000,31.800000,31.800000,61.000000,3.877953,13.322500,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,13479.000000,27970.409877
